# Task 5: Strategy Backtesting

## Objective
Validate the portfolio strategy by simulating its performance on historical data (2025-2026).

## Benchmark
- 60% SPY
- 40% BND

## Strategy
- Use optimal weights from Task 4 (assumed here).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src to path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_loader import fetch_data, clean_data

# Load Data
data_path = "../data/processed/historical_data.csv"
if os.path.exists(data_path):
    data = pd.read_csv(data_path, index_col=0, parse_dates=True, header=[0, 1])
else:
    assets = ['TSLA', 'BND', 'SPY']
    data = clean_data(fetch_data(assets, '2015-01-01', '2026-01-15'))

try:
    prices = data.xs('Close', axis=1, level=1)
except:
    prices = data

prices.dropna(inplace=True)

# Define Backtest Period
start_date = '2025-01-01'
end_date = '2026-01-15'
backtest_data = prices.loc[start_date:end_date]
backtest_returns = backtest_data.pct_change().dropna()

backtest_returns.head()

## 1. Define Weights
- **Benchmark**: 60% SPY, 40% BND, 0% TSLA
- **Strategy**: (Example) 50% TSLA, 30% SPY, 20% BND (derived from Task 4)

In [ ]:
benchmark_weights = np.array([0.0, 0.4, 0.6]) # TSLA, BND, SPY - Verify column order!
print("Assets:", backtest_returns.columns)
# Ensure weights align with columns
# Assuming columns are sorted or consistent: ['BND', 'SPY', 'TSLA'] or similar.
# Let's map explicitly.

weights_dict_benchmark = {'TSLA': 0.0, 'BND': 0.4, 'SPY': 0.6}
weights_dict_strategy = {'TSLA': 0.5, 'BND': 0.2, 'SPY': 0.3} # Example Optimal

# Create weight vectors
asset_order = backtest_returns.columns
w_bench = np.array([weights_dict_benchmark[asset] for asset in asset_order])
w_strat = np.array([weights_dict_strategy[asset] for asset in asset_order])

print("Benchmark Weights:", w_bench)
print("Strategy Weights:", w_strat)

## 2. Simulate Performance
Assumes daily rebalancing (simplest calculation: returns * weights).
For "Hold" strategy without rebalancing, we'd need to track share counts.
Here we assume a simple "Daily Rebalanced" index or just daily portfolio return.
For "Hold" (Drifting Weights), we calculate cumulative value.

In [ ]:
# 1. Simple Daily Rebalancing Return
port_ret_bench = backtest_returns.dot(w_bench)
port_ret_strat = backtest_returns.dot(w_strat)

# Cumulative Returns
cum_ret_bench = (1 + port_ret_bench).cumprod()
cum_ret_strat = (1 + port_ret_strat).cumprod()

# Plot
plt.figure(figsize=(12, 6))
plt.plot(cum_ret_bench, label='Benchmark (60/40)')
plt.plot(cum_ret_strat, label='Strategy (Model-Based)')
plt.title('Cumulative Returns: Strategy vs Benchmark')
plt.legend()
plt.show()

## 3. Metrics Analysis

In [ ]:
def calculate_metrics(returns):
    total_return = (1 + returns).prod() - 1
    annual_return = returns.mean() * 252
    volatility = returns.std() * np.sqrt(252)
    sharpe = (annual_return - 0.04) / volatility # Assumed 4% RF
    
    # Max Drawdown
    cum_returns = (1 + returns).cumprod()
    peak = cum_returns.expanding(min_periods=1).max()
    drawdown = (cum_returns / peak) - 1
    max_drawdown = drawdown.min()
    
    return {
        "Total Return": total_return,
        "Annual Return": annual_return,
        "Volatility": volatility,
        "Sharpe Ratio": sharpe,
        "Max Drawdown": max_drawdown
    }

metrics_bench = calculate_metrics(port_ret_bench)
metrics_strat = calculate_metrics(port_ret_strat)

comparison = pd.DataFrame([metrics_bench, metrics_strat], index=['Benchmark', 'Strategy'])
print(comparison)